In [55]:
import rasterio
from rasterio.merge import merge
import glob
import os
import requests
import zipfile

def download_and_extract(url, file_path, output_folder):
    """Download a ZIP file from a URL, extract its contents, and delete the ZIP file."""
    if not os.path.exists(file_path):
        try:
            print(f"Downloading from {url}...")
            response = requests.get(url, verify=False)  # Bypass SSL verification
            response.raise_for_status()  # Raise an error for bad responses
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded and saved to {file_path}")

            # Extract the ZIP file
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(output_folder)  # Extract to the output folder
            print(f"Extracted {file_path}")

            # Remove the ZIP file after extraction
            os.remove(file_path)
            print(f"Deleted the ZIP file: {file_path}")

        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")
        except zipfile.BadZipFile as e:
            print(f"Failed to extract {file_path}: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")



def merge_tif_files(input_folder, output_file, file_prefix, nodata_value=-9999):
    """Merge TIF files in a folder with a specific prefix into a single raster file.
    ------
    Input:
    - input_folder (str): Path to the folder containing the TIF files.
    - output_file (str): Path to the output raster file.
    - file_prefix (str): Prefix of files to merge ("M_" for DTM or "R_" for DSM).
    - nodata_value (int, optional): Value to replace the nodata value. Defaults to -9999.

    Output:
    - tile_bounds (list): List containing the name of the tile and the extent of that tile, for downloading the buidling data.
    - The function writes the merged file directly to the specified `output_file`.
    """
    # Find TIF files that match the given prefix
    tif_files = glob.glob(os.path.join(input_folder, f'{file_prefix}*.TIF'))

    tile_bounds = []

    if not tif_files:
        print(f"No TIF files found for prefix {file_prefix}.")
        return

    src_files_to_mosaic = []

    for tif_file in tif_files:
        src = rasterio.open(tif_file)

        # Extract the tile name & 
        base_name = os.path.basename(tif_file)
        tile_name = base_name.split('_', 1)[-1].split('.')[0]
        tile_bounds.append([tile_name, src.bounds])

        src_files_to_mosaic.append(src)

    mosaic, out_transform = merge(src_files_to_mosaic)

    # Replace existing nodata values with new nodata value
    for src in src_files_to_mosaic:
        if 'nodata' in src.meta:
            mosaic[mosaic == src.nodata] = nodata_value

    out_meta = src_files_to_mosaic[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform,
        "count": mosaic.shape[0],
        "nodata": nodata_value
    })

    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    print(f"Merged {len(tif_files)} TIF files with prefix {file_prefix} into {output_file}")

    # close & delete original TIF files & save bounds
    for src in src_files_to_mosaic:
        src.close()
    for tif_file in tif_files:
        os.remove(tif_file)
        print(f"Deleted original TIF file: {tif_file}")
        
    return tile_bounds


def download_raster_tiles(tile_list_file, output_folder, name):
    """
    Download DSM and DTM files for each tile specified in a text file.
    ------
    Input:
    - tile_list_file (str): Path to the text file containing the list of tiles to download.
    - output_folder (str): Directory where the downloaded and unzipped files will be saved.
    - name (str): Name of the output raster file.
    Output:
    - tile_bounds (list): List containing the name of the tile and the extent of that tile, for downloading the buidling data.
    - The function writes output files directly to the specified `output_folder`.
    """

    # Base URLs for downloading DTM and DSM tiles
    base_url_dtm = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m"
    base_url_dsm = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m"

    # Read tile names from the text file
    with open(tile_list_file, 'r') as f:
        tile_names = [line.strip() for line in f.readlines() if line.strip()]

    os.makedirs(output_folder, exist_ok=True)

    for tile_name in tile_names:
        # Construct the URLs for DTM and DSM
        # dtm_url = f"{base_url_dtm}/M_{tile_name}.zip"
        dsm_url = f"{base_url_dsm}/R_{tile_name}.zip"

        # Define file paths for downloaded zip files
        # dtm_file_path = os.path.join(output_folder, f"M_{tile_name}.zip")
        dsm_file_path = os.path.join(output_folder, f"R_{tile_name}.zip")

        # Download and extract DTM and DSM files
        # download_and_extract(dtm_url, dtm_file_path, output_folder)
        download_and_extract(dsm_url, dsm_file_path, output_folder)

    # Merge all DTM and DSM files separately after all downloads are done
    # merged_dtm_output_file = os.path.join(output_folder, f"{name}_DTM.TIF")
    merged_dsm_output_file = os.path.join(output_folder, f"{name}_DSM.TIF")

    # Merge files starting with "M_" for DTM and "R_" for DSM
    # merge_tif_files(output_folder, merged_dtm_output_file, "M_")
    tile_bounds = merge_tif_files(output_folder, merged_dsm_output_file, "R_")

    print(f"All DTM and DSM files processed and merged.")
    
    return tile_bounds


In [56]:
tile_bounds = download_raster_tiles("C:/Geomatics/shady_amsterdam/Calculating shade/Calculating_chm_dsm/ams_tiles_test.txt", "testingbounds", "deletethis")

C:\Users\jessi\anaconda3\envs\synthesis\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns_hwh.fundaments.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded and saved to testingbounds\R_25DN1.zip
Extracted testingbounds\R_25DN1.zip
Deleted the ZIP file: testingbounds\R_25DN1.zip


C:\Users\jessi\anaconda3\envs\synthesis\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns_hwh.fundaments.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded and saved to testingbounds\R_25DN2.zip
Extracted testingbounds\R_25DN2.zip
Deleted the ZIP file: testingbounds\R_25DN2.zip
Merged 2 TIF files with prefix R_ into testingbounds\deletethis_DSM.TIF
Deleted original TIF file: testingbounds\R_25DN1.TIF
Deleted original TIF file: testingbounds\R_25DN2.TIF
All DTM and DSM files processed and merged.


In [57]:
import os
import glob
import rasterio
import geopandas as gpd
from owslib.wfs import WebFeatureService
from shapely.geometry import box

In [13]:
print(tile_bounds)

[['25DN1', BoundingBox(left=110000.0, bottom=481250.0, right=115000.0, top=487500.0)], ['25DN2', BoundingBox(left=115000.0, bottom=481250.0, right=120000.0, top=487500.0)]]


In [58]:
import requests
import geopandas as gpd
import pandas as pd
import gzip
from io import BytesIO

def download_wfs_data(wfs_url, layer_name, bbox, output_gpkg, tile_name):
    """
    Download data from a WFS server in batches and save it to a GeoPackage.
    -----------------------------------------------------
    Input:
    -   wfs_url (str): URL of the WFS service.
    -   layer_name (str): The layer name to download.
    -   bbox (tuple): Bounding box as (minx, miny, maxx, maxy).
    -   output_gpkg (str): Path to the output GeoPackage file.
    -   tile_name (str): Layer name for saving in the GeoPackage.
    Output:
    -   None: saves a GeoPackage file to the given {output_gpkg} at layer {tile_name}.
    """
    # Initialize variables for feature collection, max requestable amount from server is 10000
    all_features = []
    start_index = 0
    count = 10000  

    while True:
        params = {
            "SERVICE": "WFS",
            "REQUEST": "GetFeature",
            "VERSION": "2.0.0",
            "TYPENAMES": layer_name,
            "SRSNAME": "urn:ogc:def:crs:EPSG::28992",
            "BBOX": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]},urn:ogc:def:crs:EPSG::28992",
            "COUNT": count,
            "STARTINDEX": start_index
        }

        # Mimicking a QGIS request
        headers = {
            "User-Agent": "Mozilla/5.0 QGIS/33411/Windows 11 Version 2009"
        }

        response = requests.get(wfs_url, params=params, headers=headers)

        # Check if the request was successful & donwload data 
        if response.status_code == 200:
            if response.headers.get('Content-Encoding', '').lower() == 'gzip' and response.content[:2] == b'\x1f\x8b':
                data = gzip.decompress(response.content)
            else:
                data = response.content

            with BytesIO(data) as f:
                gdf = gpd.read_file(f)

            all_features.append(gdf)

            # Check if the number of features retrieved is less than the requested count: then we can stop
            if len(gdf) < count:
                break  

            # Start index for next request
            start_index += count

        else:
            print(f"Failed to download WFS data. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            break  
        
    # Concatenate all features into a single GeoDataFrame
    if all_features:
        full_gdf = gpd.GeoDataFrame(pd.concat(all_features, ignore_index=True))

        # Saving to the GeoPackage with tile name
        full_gdf.to_file(output_gpkg, layer=tile_name, driver="GPKG")
        print(f"Downloaded and saved layer '{tile_name}' to {output_gpkg}")
    else:
        print("No features were downloaded.")

In [ ]:
wfs_url = "https://data.3dbag.nl/api/BAG3D/wfs"
output_gpkg = "amsterdam"
layer_name = "BAG3D:lod13"
for i in range(len(tile_bounds)):
    tile_name = tile_bounds[i][0]
    bbox = tile_bounds[i][1]
    bbox_tuple = (bbox.left, bbox.bottom, bbox.right, bbox.top)
    print(bbox_tuple)

    download_wfs_data(
        wfs_url=wfs_url,
        layer_name=layer_name,
        bbox=bbox_tuple,
        output_gpkg=output_gpkg,
        tile_name=tile_name
    )

(110000.0, 481250.0, 115000.0, 487500.0)
Downloaded and saved layer '25DN1' to amsterdam
(115000.0, 481250.0, 120000.0, 487500.0)


In [30]:
import rasterio